<a href="https://colab.research.google.com/github/microsoft/FLAML/blob/main/notebook/autogen_agent_web_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interactive LLM Agent Dealing with Web Info

FLAML offers an experimental feature of interactive LLM agents, which can be used to solve various tasks with human or automatic feedback, including tasks that require using tools via code.
Please find documentation about this feature [here](https://microsoft.github.io/FLAML/docs/Use-Cases/Auto-Generation#agents-experimental).

In this notebook, we demonstrate how to use `AssistantAgent` and `UserProxyAgent` to perform tasks which require acquiring info from the web:
* discuss a paper based on its URL.
* discuss about stock market.

Here `AssistantAgent` is an LLM-based agent that can write Python code (in a Python coding block) for a user to execute for a given task. `UserProxyAgent` is an agent which serves as a proxy for a user to execute the code written by `AssistantAgent`. By setting `human_input_mode` properly, the `UserProxyAgent` can also prompt the user for feedback to `AssistantAgent`. For example, when `human_input_mode` is set to "ALWAYS", the `UserProxyAgent` will always prompt the user for feedback. When user feedback is provided, the `UserProxyAgent` will directly pass the feedback to `AssistantAgent` without doing any additional steps. When no user feedback is provided, the `UserProxyAgent` will execute the code written by `AssistantAgent` directly and return the execution results (success or failure and corresponding outputs) to `AssistantAgent`.

## Requirements

FLAML requires `Python>=3.8`. To run this notebook example, please install flaml with the [autogen] option:
```bash
pip install flaml[autogen]
```

In [1]:
# %pip install flaml[autogen]~=2.0.0rc4

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/FLAML/docs/reference/autogen/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.


In [2]:
from flaml import oai

config_list = oai.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt4", "gpt-4-32k", "gpt-4-32k-0314"],
    },
)

It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). Only the models with matching names are kept in the list based on the filter condition.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4-32k',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt4',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k-0314',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choose "upload file" icon.

You can set the value of config_list in other ways you prefer, e.g., loading from a YAML file.

## Example Task: Investment suggestion with realtime data

We invoke the `initiate_chat()` method of the user proxy agent to start the conversation. When you run the cell below, you will be prompted to provide feedback after the assistant agent sends a "TERMINATE" signal in the end of the message. If you don't provide any feedback (by pressing Enter directly), the conversation will finish. Before the "TERMINATE" signal, the user proxy agent will try to execute the code suggested by the assistant agent on behalf of the user.

In [1]:
from flaml import oai
import asyncio

def get_market_news(ind, ind_upper):
    import requests
    import json
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    # url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&sort=LATEST&limit=5&apikey=demo'
    # r = requests.get(url)
    # data = r.json()
    with open('market_news_local.json', 'r') as file:
        # Load JSON data from file
        data = json.load(file)
    feeds = data["feed"][10+ind:10+ind_upper]
    feeds_summary = ("\n").join([ "News summary:" + f["title"] + f["summary"] + \
    ". The Overall_sentiment_score of this news is:" + str(f["overall_sentiment_score"])\
    # + "Ticker sentiment:" + str(f["ticker_sentiment"]) 
    for f in feeds])
    return feeds_summary

async def run_async():
    from flaml.autogen.agent import AssistantAgent, UserProxyAgent

    config_list = oai.config_list_from_models(key_file_path="./", model_list=["gpt-3.5-turbo"], exclude="aoai")
    # create an AssistantAgent instance named "assistant"
    assistant = AssistantAgent(
        name="assistant",
        oai_config={
            "request_timeout": 600,
            "seed": 41,
            "config_list": config_list,
            "model": "gpt-3.5-turbo",  # modify if the endpoint you use doesn't support this model
            "temperature": 0,
        }
    )
    # create a UserProxyAgent instance named "user"
    user = UserProxyAgent(
        name="user",
        human_input_mode="TERMINATE",
        max_consecutive_auto_reply=5,
        is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
        code_execution_config={"work_dir": "web", "use_docker": False},
    )

    chat_task = asyncio.create_task(user.a_initiate_chat(
    assistant,
    message="""Give me investment suggestion on Apple and Microsoft. """,
    ))

    async def add_stock_price_data():
        for i in range(0, 5, 1):
            latest_news = get_market_news(i, i+1)
            await user.a_add(f"Just got several latest market news. Answer my question based on them. \n {latest_news}")
            await asyncio.sleep(5)

    data_task = asyncio.create_task(add_stock_price_data())
    await chat_task
    await data_task

await run_async()

user (to assistant):

Give me investment suggestion on Apple and Microsoft. 

--------------------------------------------------------------------------------
Ding! New data added!!
New data addition waited!
assistant (to user):

To provide investment suggestions on Apple and Microsoft, we need to gather some financial data about these companies. We can use the yfinance library in Python to fetch the required information. Please execute the following Python code to get the latest stock information for Apple and Microsoft:

```python
# filename: stock_info.py
import yfinance as yf

# Get the stock information for Apple
apple = yf.Ticker("AAPL")
apple_info = apple.info
apple_price = apple_info['regularMarketPrice']
apple_change = apple_info['regularMarketChange']
apple_percent_change = apple_info['regularMarketChangePercent']

# Get the stock information for Microsoft
microsoft = yf.Ticker("MSFT")
microsoft_info = microsoft.info
microsoft_price = microsoft_info['regularMarketPrice']
micr